In [43]:
import os
import subprocess
import time
import glob
from astropy.io import fits

In [2]:
path = '/Users/jiaxuanli/Research/SideProjects/low-z-galaxy-survey/'

In [3]:
os.chdir(path + 'sweep/')
filename = glob.glob('*')
os.chdir('..')

In [4]:
os.getcwd()

'/Users/jiaxuanli/Research/SideProjects/low-z-galaxy-survey'

## Match the sweep files with HSC catalog, with precision = 2 arcsec. Then save them to `matched` folder.

In [6]:
since = time.time()

precision = 2 #arcsec

for name in filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 in1=dr2_s18a_wide_specz.fits in2=./sweep/'+name+' out=./matched/match2-'+name+' matcher=sky values1="specz_ra specz_dec" values2="ra dec" params="'+str(precision)+'"', shell=True)
    p.wait()
    
time_elapsed = time.time() - since
print('Matching complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # Print time  
p.terminate()

Matching complete in 10m 3s


## Delete those empty files

In [72]:
def file_size(file_path):
    """
    this function will return the file size in KB
    """
    if os.path.isfile(file_path):
        file_info = os.stat(file_path)
        return file_info.st_size/1024
    
os.chdir(path + 'matched/')
matched_filename = glob.glob('*')
# Lets check the file size of MS Paint exe 
# or you can use any file path
for name in matched_filename:
    size = file_size(name)
    if size < 50:
        os.remove(name)

## Save the matched files' name to a `.txt` file

In [29]:
os.chdir(path + 'matched/')
matched_filename = glob.glob('*')

os.chdir('..')
fh = open('matched-filename.txt', 'w')
for name in matched_filename:
    fh.write(path + 'matched/' + name)
    fh.write('\n')
fh.close()

In [30]:
len(matched_filename)

49

## Find out those galaxies no contained in HSC catalog, and save then into `not-matched` folder.

In [27]:
since = time.time()

precision = 2 #arcsec

for name in matched_filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 join=2not1 in1=dr2_s18a_wide_specz.fits in2=./sweep/'+name[7:]+' out=./not-matched/not-match2-'+name[7:]+' matcher=sky values1="specz_ra specz_dec" values2="ra dec" params="'+str(precision)+'"', shell=True)
    p.wait()
    
time_elapsed = time.time() - since
print('Matching complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # Print time   
p.terminate()

Matching complete in 3m 14s


## Save not-matched files' name to a `.txt` file

In [33]:
# Generate not-matched filenames #
os.chdir(path + 'not-matched/')
not_matched_filename = glob.glob('*')
os.chdir('..')
fh = open('not-matched-filename.txt', 'w')
for name in not_matched_filename:
    fh.write(path + 'not-matched/' + name)
    fh.write('\n')
fh.close()

In [34]:
len(not_matched_filename)

49

## Match these `not-matched` files with GAMA catalog

In [35]:
since = time.time()
precision = 2 #arcsec

for name in not_matched_filename:
    p = subprocess.Popen('java -jar stilts.jar tmatch2 in1=SpecObjv27.fits in2=./not-matched/'+name+' out=./GAMA/GAMA-match2-'+name+' matcher=sky values1="RA DEC" values2="ra dec" params="'+str(precision)+'"', shell=True)
    p.wait()
    
time_elapsed = time.time() - since
print('Matching complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60)) # Print time   
p.terminate()

Matching complete in 1m 7s


In [36]:
# Generate GAMA-matched filenames #
os.chdir(path + 'GAMA/')
GAMA_matched_filename = glob.glob('*')
os.chdir('..')
fh = open('GAMA-matched-filename.txt', 'w')
for name in GAMA_matched_filename:
    fh.write(path + 'GAMA/' + name)
    fh.write('\n')
fh.close()

## Concatenate HSC-matched files

In [37]:
os.chdir(path)
os.getcwd()

'/Users/jiaxuanli/Research/SideProjects/low-z-galaxy-survey'

In [38]:
p = subprocess.Popen('java -jar stilts.jar tcat in=@matched-filename.txt out=HSC-match2-combined.fits', shell=True)

## Concatenate GAMA-matched files

In [39]:
os.chdir(path)
os.getcwd()
p = subprocess.Popen('java -jar stilts.jar tcat in=@GAMA-matched-filename.txt out=GAMA-match2-combined.fits', shell=True)

## The following processes are convenient using TOPCAT instead of STILS.
- Show columns of `GAMA-match2-combined.fits`, and unselect items behind 19 (`URL_IMG`), but keep `RA_2` and `DEC_2`.
- Use `Join->Concatenate Tables` in TOPCAT, and assign `RA -> RA_2`, `DEC -> DEC_2`, then concatenate.
- Then match the concatenated table with `GAMA-match2-combined.fits`, but select `Join type = all from 1`. 
- In the matched table, unselect column `RA_2, DEC_2, seperation, seperation_1`.
- Save this table.
- You get it! :)